In [29]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geemap as geemap
import geemap.chart as chart

In [3]:
ee.Authenticate()

Enter verification code:  4/1AVHEtk6R9nX4C-OGrkDHfyzfIdEJe1H20Jo-QJTKZJPNpfrQXPHegiH1Bz4



Successfully saved authorization token.


In [92]:
geemap.ee_initialize()
ee.Initialize()

# Data collection

In [93]:
aoi = ee.Geometry.Polygon([[[102.4894322423133, 0.8283349585410177],
                            [102.4894322423133, 0.16918388171760845],
                            [103.2639683751258, 0.16918388171760845],
                            [103.2639683751258, 0.8283349585410177]]])
start = '2020-02-01'
end = '2020-03-15'

s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD').filter(ee.Filter.eq('instrumentMode',
                                                                            'IW')).filter(ee.Filter.eq('orbitProperties_pass',
                                                                                                       'DESCENDING')).filterDate(start, end).filterBounds(aoi)
s1_image = s1_collection.first().clip(aoi)

In [ ]:
# Data Visualization

In [94]:
visualisation_params = {'min': -25, 'max': 0, 'palette' : ['black', 'white']}

s1VV_image = s1_image.select('VV')
s1VH_image = s1_image.select('VH')

def VV_VH_calc(image):
    VV_VH = image.select('VV').subtract(image.select('VH')).rename('VVVH')
    return image.addBands([VV_VH])

s1_image = VV_VH_calc(s1_image)

Map = geemap.Map()

Map.addLayer(s1_image.select('VH'), vis_params = visualisation_params, name = 'Sentinel 1 - VH')
Map.addLayer(s1_image.select('VV'), vis_params = visualisation_params, name = 'Sentinel 1 - VV')
Map.addLayer(s1_image.select('VVVH'), vis_params = visualisation_params, name = 'Sentinel 1 - VVVH')

visualisation_params = {"opacity": 1,
                        "bands": ["VV","VH","VVVH"],
                        "min": [-25,-30,0],
                        "max": [-5,-10,10],
                        "gamma": [1,1,1]}

Map.addLayer(s1_image, visualisation_params, name = 'Sentinel 1 - Composite')

Map.centerObject(aoi, 9)

Map

Map(center=[0.4987653109314045, 102.87670030871976], controls=(WidgetControl(options=['position', 'transparent…

In [86]:
def get_Histo(img, band, aoi):
    """
        Function to calculate the distribution of values in one band of an image.
        Inputs:
            - img: ee.Image object
            - band: String with the name of the band
            - aoi: Area Of Interest
    """
    
    sample = img.select(band).sample(aoi, 100)
    histo = chart.feature_histogram(sample, property = band, xlabel = 'Signal [dB]', ylabel = 'Pixel count', title = 'Histogram for ' + band + ' band.',colors = ['#1d6b99'])
    return histo

get_Histo(s1_image, 'VV', aoi)
get_Histo(s1_image, 'VH', aoi)
get_Histo(s1_image, 'VVVH', aoi)

In [95]:
s1_collection